In [1]:
import sys
import os

# Get the current working directory
# Get project root (folder above 'notebooks')
project_root = os.path.dirname(os.getcwd())

# Path to scripts folder
scripts_path = os.path.join(project_root, "scripts")

# Add to sys.path
sys.path.append(project_root)
sys.path.append(scripts_path)
# Add db folder
db_path = os.path.join(project_root, "db")
sys.path.append(db_path)
import pandas as pd

In [2]:
from connection import engine
from analysis import perform_thematic_analysis,assign_themes_to_reviews,identify_drivers_and_pain_points,get_theme_sentiment_distribution

Set NLTK data path to: C:\Users\hp\.nltk_data


In [8]:
sentiment_sql = """
SELECT r.review_text AS review,
r.rating,
r.review_date AS date,
b.bank_name AS bank,
r.source,
r.sentiment_score AS compound_score,
r.sentiment_label
FROM reviews r
JOIN banks b on  r.bank_id = b.bank_id ; 
"""
all_review = pd.read_sql(sentiment_sql, engine)
all_review.head()

2025-12-01 17:42:17,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-01 17:42:17,843 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-12-01 17:42:17,848 INFO sqlalchemy.engine.Engine [cached since 1558s ago] {'table_name': '\nSELECT r.review_text AS review,\nr.rating,\nr.review_date AS date,\nb.bank_name AS bank,\nr.source,\nr.sentiment_score AS compound_score,\nr.sentiment_label\nFROM reviews r\nJOIN banks b on  r.bank_id = b.bank_id ; \n', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-12-01 17:42:17

,review,rating,date,bank,source,compound_score,sentiment_label
0,CBE ይለያል።,5,2025-11-29,CBE,com.combanketh.mobilebanking,0.00,NEUTRAL
1,it's special for me,5,2025-11-29,CBE,com.combanketh.mobilebanking,0.40,POSITIVE
2,Make it user friendly.,2,2025-11-29,CBE,com.combanketh.mobilebanking,0.49,POSITIVE
3,maaliif daddafee install gaafata,3,2025-11-28,CBE,com.combanketh.mobilebanking,0.00,NEUTRAL
4,good app,5,2025-11-28,CBE,com.combanketh.mobilebanking,0.44,POSITIVE


In [10]:
themed_keywords_df, keyword_to_theme_map =  perform_thematic_analysis(all_review)


--- Starting Thematic Analysis (Keyword Extraction using TF-IDF) ---
Extracted 50 top keywords/n-grams.

--- Thematic Grouping Logic (Documentation) ---
Keywords were grouped into 5 themes based on keyword matching:
 - Account Access & Stability: Matches keywords like 'login, fingerprint, cant, app crashed...'
 - Transaction Performance: Matches keywords like 'transfer, slow, pending, takes long...'
 - User Interface & Experience: Matches keywords like 'interface, design, ui, easy use...'
 - Customer Support & Service: Matches keywords like 'customer service, branch, help, contact...'
 - Feature Requests & Missing Functionality: Matches keywords like 'feature, need, add, option...'

Top Keywords and Their Assigned Themes:
   keyword_phrase               assigned_theme
0             app                Other/General
1            good                Other/General
2            best                Other/General
3            nice                Other/General
4            bank               

In [11]:
themed_sentiment_df = assign_themes_to_reviews(all_review, keyword_to_theme_map)


--- Starting Theme Assignment to Individual Reviews ---
Theme assignment complete. Added 'identified_themes' column.


In [12]:
#Identify Drivers and Pain Points (Top 3)
driver_pain_point_results = identify_drivers_and_pain_points(themed_sentiment_df, top_n=3)


--- Identifying Top Drivers and Pain Points Per Bank ---


Analyzing Drivers & Pain Points: 100%|██████████| 3/3 [00:00<00:00, 108.36it/s]

Driver and Pain Point analysis complete.


In [13]:
print("\n" + "="*70)
print("🎯 TOP 3 DRIVERS AND PAIN POINTS PER BANK (THEMATIC ANALYSIS)")
print("="*70)

for bank, results in driver_pain_point_results.items():
    print(f"\n--- {bank} ---")
    print("  ✅ Drivers (Positive Sentiment):")
    if results['drivers']:
        for d in results['drivers']:
            print(f"    - {d}")
    else:
        print("    - No significant drivers identified.")
        
    print("  ❌ Pain Points (Negative Sentiment):")
    if results['pain_points']:
        for p in results['pain_points']:
            print(f"    - {p}")
    else:
        print("    - No significant pain points identified.")

print("="*70)


🎯 TOP 3 DRIVERS AND PAIN POINTS PER BANK (THEMATIC ANALYSIS)

--- CBE ---
  ✅ Drivers (Positive Sentiment):
    - User Interface & Experience (18 mentions)
    - Transaction Performance (13 mentions)
    - Feature Requests & Missing Functionality (12 mentions)
  ❌ Pain Points (Negative Sentiment):
    - User Interface & Experience (5 mentions)
    - Feature Requests & Missing Functionality (5 mentions)
    - Transaction Performance (5 mentions)

--- BoAMobile ---
  ✅ Drivers (Positive Sentiment):
    - Transaction Performance (15 mentions)
    - Feature Requests & Missing Functionality (14 mentions)
    - User Interface & Experience (9 mentions)
  ❌ Pain Points (Negative Sentiment):
    - Transaction Performance (12 mentions)
    - User Interface & Experience (10 mentions)
    - Feature Requests & Missing Functionality (10 mentions)

--- DashenBank ---
  ✅ Drivers (Positive Sentiment):
    - User Interface & Experience (34 mentions)
    - Feature Requests & Missing Functionality (24 m